***RAHUL VIMALKANTH***

# IMPORTING LIBRARIES AND INPUT FILES

In [17]:
# Install necessary libraries
%pip -qq install groq
%pip -qq install nltk rouge-score
%pip install gradio -q

In [18]:
# Import required modules
import os
from google.colab import userdata
from groq import Groq
import gradio as gr
import json
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

In [111]:
# Initialize Groq client with API key from user data
client = Groq(api_key=userdata.get('GROQ_API_KEY'))
MODEL1 = "llama3-groq-70b-8192-tool-use-preview"
MODEL2 = "llama3-groq-70b-8192-tool-use-preview"

The *tools.json* contains the tools provided in the problem statement. The *examples.json* file has the examples provided in the problem statement. The *generated_data_examples.json* contains high quality queries and solutions that I generated using another LLM.

In [67]:
# Function to load JSON data from a file
def load_json_file(file_path):
    try:
        with open(file_path, 'r') as f:
            return json.load(f)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

tools = load_json_file("tools.json")
examples = load_json_file("examples.json")
generated_data_examples = load_json_file("generated_data_examples.json")

# CREATING PROMPT TEMPLATE FOR THE MODEL

In [115]:
output_format = {
"type": "array",
"items": {
"type": "object",
"properties": {
"tool_name": {
"type": "string"
},
"arguments": {
"type": "array",
"items": {
"type": "object",
"properties": {
"argument_name": {
"type": "string"
},
"argument_value": {
"type": ["string", "array", "object", "boolean", "number"]
}
},
"required": [
"argument_name",
"argument_value"
]
}
}
},
"required": [
"tool_name",
"arguments"
]
}
}

In [116]:
# Function to create a prompt for MODEL1
def prompt_template_model1(query, tools, output_format, examples):
    return f"""
<|start_header_id|>system<|end_header_id|>

You are a query solver. Using the provided tools, your task is to solve the query by selecting and calling the relevant tools in sequence.

### Key Instructions:
1) Carefully read the tool and argument descriptions to understand what outputs a tool generates and what inputs it requires.
2) Use the output of previous tool calls as input for the current tool, referenced as $$PREV[i], where `i` is the index of the previous tool call.
3) Always check if authentication tools like "who_am_i", "team_id", or "get_sprint_id" are necessary. Do not use more than 4 tools.
4) Handle the "type" argument in "works_list" correctly (e.g., "issue", "ticket", or "task").
5) Stop once the task is complete, and no further tools are needed to solve the query.
6) Only use the tools provided. If the query cannot be solved with the given tools, return an empty list: [].
7) Understand the solutions for the given queries in the examples provided.

### Available Tools:
<tools>
{tools}
</tools>

<|eot_id|><|start_header_id|>user<|end_header_id|>

Analyze the user query and select only the relevant tools. Reference the output of previous tools using $$PREV[i], and generate a sequence of tool calls that will solve the query.Follow the JSON schema provided in <output_format></output_format>.

If any tool has an empty arguments list, do not include it. Return only the JSON output without explanations. If the query cannot be solved with the available tools, return an empty list: [].

### JSON Output Format.Again strictly follow the given output format:
<output_format>
{output_format}
</output_format><|eot_id|><|start_header_id|>user<|end_header_id|>

### Examples with queries and their corresponging tool calls.
{examples}
<|eot_id|><|start_header_id|>tool<|end_header_id|>
    """.strip()


In [117]:
# Function to create a prompt for MODEL2
def prompt_template_model2(query, model1_output, generated_data_examples, output_format):
    """Create a concise and efficient prompt for MODEL2 to refine the output of MODEL1."""
    return f"""
<|start_header_id|>system<|end_header_id|>

You are an advanced AI tasked with refining the tool calls generated by a previous model (MODEL1). Your goal is to optimize the output to directly address the user's query using only the necessary tools and accurate arguments.

### Your Task:
1. **User Query**: Solve this query using the provided tools.
2. **MODEL1 Output**: Review and correct the tool calls generated by MODEL1, improving them as needed.
3. **Examples**: Refer to these examples for guidance on how to correct tool calls and arguments.
4. **Generated Examples**: Search these examples for queries which are similar to user query. Understand the solutions for those queries and try to answer the user query.

### Requirements:
- Only include tools that are strictly relevant to solving the query.
- All arguments must be accurately filled and non-empty.
- Remove any unnecessary, redundant, or incorrect tool calls.
- Ensure output is valid JSON, strictly following the format below.

### Instructions:
1. **Optimize Tool Selection**: Only include tools that are essential. Omit any irrelevant tool calls.
2. **Ensure Argument Accuracy**: Arguments should be filled based on the query, referring to `$$PREV[i]` for dependencies from prior tools.
3. **JSON Format**: Output must adhere strictly to the following JSON format within the <output_format> tags:

### JSON Output Format:
<output_format>
{output_format}
</output_format>

### Input:
- **User Query**:
  {query}
- **MODEL1 Output**:
  {model1_output}
- **Generated Examples**:
  {generated_data_examples}

Provide your refined tool calls in the specified JSON format, ensuring no unnecessary tools or empty arguments are included.

<|eot_id|><|start_header_id|>user<|end_header_id|>



""".strip()


# PROMPTING MODEL1 AND MODEL2

In [148]:
query = "List all high severity tickets coming in from slack from customer Cust123 and generate a summary of them."
# Execute the queries and print the results

prompt1 = prompt_template_model1(query, tools, output_format,examples)

# Call MODEL1 to generate the initial tool calls
chat_completion1 = client.chat.completions.create(
    model=MODEL1,
    messages=[
        {"role": "system", "content": prompt1},
        {"role": "user", "content": query},
    ]
)

# Capture the response from MODEL1
model1_response = chat_completion1.choices[0].message.content
print("MODEL1 Output:", model1_response)


MODEL1 Output: [{'tool_name': 'search_object_by_name', 'arguments': [{'argument_name': 'query', 'argument_value': 'Cust123'}]}, {'tool_name': 'works_list', 'arguments': [{'argument_name': 'ticket_severity', 'argument_value': ['high']}, {'argument_name': 'ticket_source_channel', 'argument_value': ['slack']}, {'argument_name': 'ticket_rev_org', 'argument_value': '$$PREV[0]'}, {'argument_name': 'type', 'argument_value': ['ticket']}]}, {'tool_name': 'summarize_objects', 'arguments': [{'argument_name': 'objects', 'argument_value': '$$PREV[0]'}]}]


In [149]:
# Generate prompt for MODEL2
prompt2 = prompt_template_model2(query, model1_response, generated_data_examples, output_format)

# Call MODEL2 to refine the output using MODEL1's response and the examples
chat_completion2 = client.chat.completions.create(
  model=MODEL2,
  messages=[
      {"role": "system", "content": prompt2},
      {"role": "user", "content": query},
  ]
)

# Capture the response from MODEL2
model2_response = chat_completion2.choices[0].message.content
print("MODEL2 Refined Output:", model2_response)

MODEL2 Refined Output: {'id': 1, 'name': 'search_object_by_name', 'arguments': {'query': 'Cust123'}}
{'id': 2, 'name': 'works_list', 'arguments': {'ticket_severity': ['high'], 'ticket_source_channel': ['slack'], 'ticket_rev_org': '$$PREV[0]', 'type': ['ticket']}}
{'id': 3, 'name': 'summarize_objects', 'arguments': {'objects': '$$PREV[1]'}}


In [150]:
actual_response =[
          {
            "tool_name": "search_object_by_name",
            "arguments": [
              {
                "argument_name": "query",
                "argument_value": "Cust123"
              }
            ]
          },
          {
            "tool_name": "works_list",
            "arguments": [
              {
                "argument_name": "ticket_rev_org",
                "argument_value": [
                  "$$PREV[0]"
                ]
              },
              {
                "argument_name": "ticket_severity",
                "argument_value": [
                  "high"
                ]
              },
              {
                "argument_name": "ticket_source_channel",
                "argument_value": [
                  "slack"
                ]
              },
              {
                "argument_name": "type",
                "argument_value": [
                  "ticket"
                ]
              }
            ]
          },
          {
            "tool_name": "summarize_objects",
            "arguments": [
              {
                "argument_name": "objects",
                "argument_value": "$$PREV[1]"
              }
            ]
          }
        ]

# EVALUATING THE MODEL PERFORMANCE

In [147]:
# Function to calculate ROUGE score
def calculate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)

    return scores

# Evaluate MODEL1's response against ground truth (example response)
rouge_scores_model1 = calculate_rouge(json.dumps(actual_response), json.dumps(model1_response))
rouge_scores_model2 = calculate_rouge(json.dumps(actual_response), json.dumps(model2_response))

# Print the ROUGE scores
print(f"ROUGE Scores for MODEL1: {rouge_scores_model1}")

print(f"ROUGE Scores for MODEL2: {rouge_scores_model2}")

ROUGE Scores for MODEL1: {'rouge1': Score(precision=0.9814814814814815, recall=0.8833333333333333, fmeasure=0.9298245614035088), 'rougeL': Score(precision=0.8888888888888888, recall=0.8, fmeasure=0.8421052631578948)}
ROUGE Scores for MODEL2: {'rouge1': Score(precision=0.7948717948717948, recall=0.5166666666666667, fmeasure=0.6262626262626263), 'rougeL': Score(precision=0.6923076923076923, recall=0.45, fmeasure=0.5454545454545455)}


# UI USING GRADIO

In [151]:
def generate_response(query):

    # Assume these functions are defined with your existing logic for MODEL1 and MODEL2 processing
    prompt1 = prompt_template_model1(query, tools, output_format, examples)
    chat_completion1 = client.chat.completions.create(
        model=MODEL1,
        messages=[
            {"role": "system", "content": prompt1},
            {"role": "user", "content": query},
        ]
    )

    model1_response = chat_completion1.choices[0].message.content

    prompt2 = prompt_template_model2(query, model1_response, generated_data_examples, output_format)
    chat_completion2 = client.chat.completions.create(
        model=MODEL2,
        messages=[
            {"role": "system", "content": prompt2},
            {"role": "user", "content": query},
        ]
    )

    model2_response = chat_completion2.choices[0].message.content

    return model1_response, model2_response


In [152]:
with gr.Blocks(theme="dark", css="""
    .gr-button {background-color: #4CAF50; color: white; border: none;}
    .gr-button:hover {background-color: #45a049;}
    .gr-textbox {background-color: #333; color: white; border-color: #4CAF50;}
""") as ui:

    # Main Header
    gr.Markdown("<h1 style='color:#4CAF50; text-align:center;'>Tool Use Automation Interface</h1>")

    # Input Section
    with gr.Row():
        query_input = gr.Textbox(label="Enter your query", placeholder="Input your query here...", lines=2)

    # Submit Button
    submit_button = gr.Button("Submit", elem_id="submit_button")

    # Outputs for model responses
    with gr.Row():
        model1_output = gr.Textbox(label="MODEL1 Output", placeholder="Model1 output will appear here", lines=4)
        model2_output = gr.Textbox(label="MODEL2 Refined Output", placeholder="Model2 refined output will appear here", lines=4)

    # Submit action to trigger model generation
    submit_button.click(fn=generate_response, inputs=[query_input], outputs=[model1_output, model2_output])

# Launch the interface
ui.launch()

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1025: UserWarning: Cannot load dark. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/dark (Request ID: Root=1-67129284-472ce72d46fed8da59d903a2;8eb95bc2-a270-497d-993b-ea1653c5b421)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e542e6fad7561282bb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
